In [16]:
#данные уже готовы
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import make_scorer

import warnings
 
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [35]:
first_popularity_steel = pd.read_csv('first_popularity_steel.csv', sep=',')
second_popularity_steel = pd.read_csv('second_popularity_steel.csv', sep=',')
# инициализируйте ваш датасет и идите вниз

In [18]:
#first_popularity_steel.info()

In [19]:
#second_popularity_steel.info()

In [28]:
def LEARN_TO_COOK_SLAG_EASY(data): # not "easy", by chaotic
    to_drop = ['химшлак последний Al2O3', 'химшлак последний CaO',
       'химшлак последний FeO', 'химшлак последний MgO',
       'химшлак последний MnO', 'химшлак последний R',
       'химшлак последний SiO2']

    X = data.drop(to_drop, axis=1)

    targets = ['химшлак последний CaO',
           'химшлак последний FeO', 'химшлак последний MgO',
           'химшлак последний MnO', 'химшлак последний R', 'химшлак последний Al2O3', 'химшлак последний SiO2']

    for i in range(len(targets)):
        best_score_target = ""
        best_score = 1
        best_target_prediction = []
    
        for t in targets:
            y = data[t] 
            reg = CatBoostRegressor(iterations=1000,learning_rate=0.1,depth=5, verbose=False, random_seed=42)
            score = cross_val_score(reg, X, y, scoring=make_scorer(mean_absolute_percentage_error, random_seed=42), cv=5).mean()      

            if score < best_score:
                best_score_target = t
                best_score = score
        best_target_prediction = CatBoostRegressor(iterations=1000,learning_rate=0.1,depth=5, verbose=False, random_seed=42).fit(X, y).predict(X)

        print(best_score_target, "\n", best_score * 100, "%")
        targets.remove(best_score_target)
        X[best_score_target] = best_target_prediction

In [29]:
LEARN_TO_COOK_SLAG_EASY(second_popularity_steel)

химшлак последний CaO 
 5.095663181714399 %
химшлак последний SiO2 
 0.4821034998546318 %
химшлак последний R 
 4.36716685668887 %
химшлак последний Al2O3 
 1.0293225167804474 %
химшлак последний MgO 
 23.644975751324456 %
химшлак последний MnO 
 2.856799802082384 %
химшлак последний FeO 
 38.03925764054294 %


![image](depending.png)

In [60]:
def PRE_PREDICT(X, y, first_to_predict, last_target):
    print("PRE_PREDICT")
    
    for target in first_to_predict:
        y_target = y[target]
        score = cross_val_score(CatBoostRegressor(iterations=1000,
                                                  learning_rate=0.1,
                                                  depth=5,
                                                  verbose=False,
                                                  random_seed=42), X, y_target, scoring=make_scorer(mean_absolute_percentage_error), cv=5).mean()
        print(target, "\n", score * 100, "%")
        predict = CatBoostRegressor(iterations=1000,learning_rate=0.1,depth=5, verbose=False, random_seed=42).fit(X, y_target).predict(X)
        X[target] = predict
    #
    print("TARGET: ", last_target)
    y_target = y[last_target]
    score = cross_val_score(CatBoostRegressor(iterations=1000,
                                                  learning_rate=0.1,
                                                  depth=5,
                                                  verbose=False,
                                                  random_seed=42), X, y_target, scoring=make_scorer(mean_absolute_percentage_error), cv=5).mean()
    print(score * 100, "%")
    predict = CatBoostRegressor(iterations=1000,learning_rate=0.1,depth=5, verbose=False, random_seed=42).fit(X, y_target).predict(X)    
    print("END")
    
    return predict

In [61]:
# now we know what to do and now we know what to hack!

def LEARN_TO_COOK_SLAG(data):
    to_drop = ['химшлак последний Al2O3', 'химшлак последний CaO',
       'химшлак последний FeO', 'химшлак последний MgO',
       'химшлак последний MnO', 'химшлак последний R',
       'химшлак последний SiO2']

    X = data.drop(to_drop, axis=1)
    data_for_dirt = data[['химшлак последний R', 'химшлак последний SiO2', 'химшлак последний CaO']]
    df_for_dirt = pd.DataFrame()
    
    df_for_dirt['химшлак последний CaO'] = PRE_PREDICT(X.copy(), data_for_dirt, ['химшлак последний R', 'химшлак последний SiO2'], 'химшлак последний CaO')
    df_for_dirt['химшлак последний SiO2'] = PRE_PREDICT(X.copy(), data_for_dirt, ['химшлак последний R', 'химшлак последний CaO'], 'химшлак последний SiO2')
    df_for_dirt['химшлак последний R'] = PRE_PREDICT(X.copy(),data_for_dirt, ['химшлак последний CaO', 'химшлак последний SiO2'], 'химшлак последний R')
    
    X['химшлак последний CaO_1'] = df_for_dirt['химшлак последний CaO']
    X['химшлак последний SiO2_1'] = df_for_dirt['химшлак последний SiO2']
    X['химшлак последний R_1'] = df_for_dirt['химшлак последний R']
    
    print("PREPROCESS NEXT STEP")

    targets = ['химшлак последний FeO', 'химшлак последний MgO',
           'химшлак последний MnO', 'химшлак последний Al2O3']

    for i in range(len(targets)):
        best_score_target = ""
        best_score = 1
        best_target_prediction = []
    
        for target in targets:
            y = data[target] 
            reg = CatBoostRegressor(iterations=1000,learning_rate=0.1,depth=5, verbose=False, random_seed=42)
            score = cross_val_score(reg, X, y, scoring=make_scorer(mean_absolute_percentage_error), cv=5).mean()      

            if score < best_score:
                best_score_target = target
                best_score = score
        best_target_prediction = CatBoostRegressor(iterations=1000,learning_rate=0.1,depth=5, verbose=False, random_seed=42).fit(X, data[best_score_target]).predict(X)

        print(best_score_target, "\n", best_score * 100, "%")
        targets.remove(best_score_target)
        X[best_score_target + "_1"] = best_target_prediction
        
    print("END PREPROCESS")
        
    targets = ['химшлак последний CaO',
       'химшлак последний FeO', 'химшлак последний MgO',
       'химшлак последний MnO', 'химшлак последний R', 'химшлак последний Al2O3', 'химшлак последний SiO2']
    
    for i in range(len(targets)):
        best_score_target = ""
        best_score = 1
        best_target_prediction = []
    
        for target in targets:
            y = data[target] 
            reg = CatBoostRegressor(iterations=1000,learning_rate=0.1,depth=5, verbose=False, random_seed=42)
            score = cross_val_score(reg, X, y, scoring=make_scorer(mean_absolute_percentage_error), cv=5).mean()      

            if score < best_score:
                best_score_target = target
                best_score = score
        best_target_prediction = CatBoostRegressor(iterations=1000,learning_rate=0.1,depth=5, verbose=False, random_seed=42).fit(X, data[best_score_target]).predict(X)

        print(best_score_target, "\n", best_score * 100, "%")
        targets.remove(best_score_target)
        X[best_score_target] = best_target_prediction
    

In [62]:
LEARN_TO_COOK_SLAG(second_popularity_steel) # подставьте ваш датасет

PRE_PREDICT
химшлак последний R 
 6.6877374205524225 %
химшлак последний SiO2 
 5.091380215708773 %
TARGET:  химшлак последний CaO
1.9078303594461838 %
END
PRE_PREDICT
химшлак последний R 
 6.6877374205524225 %
химшлак последний CaO 
 4.820114264936122 %
TARGET:  химшлак последний SiO2
1.9361472252469134 %
END
PRE_PREDICT
химшлак последний CaO 
 5.04412489857585 %
химшлак последний SiO2 
 6.177058754589297 %
TARGET:  химшлак последний R
1.8380418072969398 %
END
PREPROCESS NEXT STEP
химшлак последний Al2O3 
 11.157949154333298 %
химшлак последний MgO 
 20.76005641392012 %
химшлак последний MnO 
 29.992494306610517 %
химшлак последний FeO 
 37.23510180637977 %
END PREPROCESS
химшлак последний CaO 
 0.3468366595815489 %
химшлак последний SiO2 
 0.49424640387056623 %
химшлак последний R 
 0.6008524951597212 %
химшлак последний Al2O3 
 1.0612581855256238 %
химшлак последний MgO 
 1.7913508743460027 %
химшлак последний MnO 
 2.59026619147882 %
химшлак последний FeO 
 2.624363097913639 %


In [63]:
LEARN_TO_COOK_SLAG(first_popularity_steel)

PRE_PREDICT
химшлак последний R 
 6.26941175221978 %
химшлак последний SiO2 
 4.927408872393381 %
TARGET:  химшлак последний CaO
2.411640865717002 %
END
PRE_PREDICT
химшлак последний R 
 6.26941175221978 %
химшлак последний CaO 
 4.779902596826551 %
TARGET:  химшлак последний SiO2
2.5051860949502505 %
END
PRE_PREDICT
химшлак последний CaO 
 5.106100907089447 %
химшлак последний SiO2 
 5.963901265640378 %
TARGET:  химшлак последний R
2.2687543460338087 %
END
PREPROCESS NEXT STEP
химшлак последний Al2O3 
 10.275585447937233 %
химшлак последний MgO 
 21.330071165469004 %
химшлак последний MnO 
 27.825339686810235 %
химшлак последний FeO 
 33.24619524994634 %
END PREPROCESS
химшлак последний R 
 0.5890921888793812 %
химшлак последний CaO 
 0.8268045056540952 %
химшлак последний SiO2 
 0.900798399845224 %
химшлак последний Al2O3 
 2.9788409798513307 %
химшлак последний MgO 
 5.662685940055458 %
химшлак последний FeO 
 7.474001582612662 %
химшлак последний MnO 
 8.528872156911053 %
